In [4]:
import pandas as pd
import numpy as np
import torch

# Task 1. Dataset generation

In [582]:
raw_data = pd.read_csv('data.tsv', delimiter = '\t', usecols = ['star_rating', 'review_body'], on_bad_lines='skip')

/var/folders/w5/ns3xhsyx7gxdvc917t999gxh0000gn/T/ipykernel_56981/2640375621.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_data = pd.read_csv('/Users/libin/Desktop/usc/cs544/HW3/amazon_reviews_us_Beauty_v1_00.tsv', delimiter = '\t', usecols = ['star_rating', 'review_body'], on_bad_lines='skip')


In [583]:
raw_data['review_body'] = raw_data.review_body.astype(str)
# drop NA from raw data
raw_data_1 = raw_data.dropna(axis = 0, how = 'any')

## generate balanced dataset

In [584]:
raw_data_2 = raw_data_1.copy()
raw_data_2['star_rating'] = raw_data_2.star_rating.astype(str)
# classify and sample from each class
data_dict = {}
data_dict[1] = raw_data_2.loc[(raw_data_2['star_rating'] == '1') | (raw_data_2['star_rating'] == '2')].sample(n = 20000, replace = False)
data_dict[2] = raw_data_2.loc[raw_data_2['star_rating'] == '3'].sample(n = 20000, replace = False)
data_dict[3] = raw_data_2.loc[(raw_data_2['star_rating'] == '4') | (raw_data_2['star_rating'] == '5')].sample(n = 20000, replace = False)

In [585]:
ordered_data = pd.concat([data_dict[1], data_dict[2], data_dict[3]], ignore_index=True)

## do some preprocessing

In [586]:
data_3 = ordered_data.copy()
# use only review_body
data_3['review'] = data_3['review_body']
# remove extra whitespace
data_3['review'] = data_3['review'].replace(r"\s+", ' ', regex = True)
data_3 = data_3.drop(columns = ['review_body'])

In [587]:
# convert all characters to lower case
data_3['review'] = data_3['review'].str.lower()
# remove url
data_3['review'] = data_3['review'].replace(r"<.*?>", " ", regex=True)
# remove http
data_3['review'] = data_3['review'].replace(r"http*\S+|www*\S+", " ", regex=True)
# remove non-alphabetical characters
data_3['review'] = data_3['review'].replace(r"[^0-9a-zA-Z]", " ", regex=True)
# remove extra whitespaces
data_3['review'] = data_3['review'].replace(r"\s+", " ", regex=True)
data_3 = data_3.dropna(axis = 0, how = 'any')

In [588]:
# do contractions
import contractions
def do_contractions(text: str):
    result = []
    for word in text.split():
        result.append(contractions.fix(word))
    return " ".join(result)
data_4 = data_3.copy()

# do some customized contractions
data_4['review'] = data_4['review'].apply(do_contractions)

data_4['review'] = data_4['review'].replace(r'dont', 'do not', regex=True)

data_4['review'] = data_4['review'].replace(r'wont', 'will not', regex=True)

data_4['review'] = data_4['review'].replace(r'cant', 'can not', regex=True)

data_4['review'] = data_4['review'].replace(r'wouldnt', 'would not', regex=True)

data_4['review'] = data_4['review'].replace(r'didnt', 'did not', regex=True)

data_4['review'] = data_4['review'].replace(r'doesnt', 'does not', regex=True)

data_4['review'] = data_4['review'].replace(r'll', 'will', regex=True)

## label the three classes

In [589]:
def classify(rating):
    if rating == 1 or rating == 2:
        return 1
    elif rating == 3:
        return 2
    else:
        return 3
data_4.star_rating = data_4.star_rating.astype(int)
data_4['star_rating'] = data_4['star_rating'].apply(classify)


In [590]:
import sklearn
data_4 = sklearn.utils.shuffle(data_4)

## divide into training and testing set

In [591]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(data_4, test_size=0.2, train_size=0.8, shuffle=True)

# Task 2. Word Embedding

## (a) Load the pretrained model

In [15]:
import gensim.downloader as api
wv = api.load('word2vec-google-news-300')

In [593]:
from numpy import dot
from numpy.linalg import norm
pairs = [
    ('buy', 'purchase'),
    ('good', 'nice'),
]
for w1, w2 in pairs:
    print('For google-news model, the similarity between %r and %r is %.2f' % (w1, w2, wv.similarity(w1, w2)))

theVec = wv['king'] - wv['man'] + wv['woman']
cos_sim = dot(theVec, wv['queen'])/(norm(theVec)*norm(wv['queen']))
print('For google-news model, the similarity between \'king - man + woman\' and \'queen\' is ', cos_sim)


For google-news model, the similarity between 'buy' and 'purchase' is 0.76
For google-news model, the similarity between 'good' and 'nice' is 0.68
For google-news model, the similarity between 'king - man + woman' and 'queen' is  0.73005176


## (b) Train my own model

In [594]:
# output my data to a .txt file, for further training model
# Note: Please modify the path to generate and read the file!
thePath = '/Users/libin/Desktop/usc/cs544/HW3/'
output_data = data_4['review']
fullPath = thePath + 'test.txt'
output_data.to_csv(fullPath, index=False, header=False, sep='\n')

In [595]:
from gensim.test.utils import datapath
from gensim import utils
class MyCorpus:
    """An iterator that yields sentences (lists of str)."""

    def __iter__(self):
        corpus_path = datapath(fullPath)
        for line in open(corpus_path):
            # assume there's one document per line, tokens separated by whitespace
            yield utils.simple_preprocess(line)

In [596]:
import gensim.models

sentences = MyCorpus()
myWord2Vec = gensim.models.Word2Vec(sentences=sentences, vector_size=300, window=13, min_count=9)

In [599]:
pairs = [
    ('buy', 'purchase'),
    ('good', 'nice'),
]
for w1, w2 in pairs:
    print('For my own model, the similarity between %r and %r is %.2f' % (w1, w2, myWord2Vec.wv.similarity(w1, w2)))

theVec = myWord2Vec.wv['king'] - myWord2Vec.wv['man'] + myWord2Vec.wv['woman']
cos_sim = dot(theVec, myWord2Vec.wv['queen'])/(norm(theVec)*norm(myWord2Vec.wv['queen']))
print('For my own model, the similarity between \'king - man + woman\' and \'queen\' is ', cos_sim)

For my own model, the similarity between 'buy' and 'purchase' is 0.80
For my own model, the similarity between 'good' and 'nice' is 0.60
For my own model, the similarity between 'king - man + woman' and 'queen' is  0.25065425


# Task 3. Simple Models

## (a) Prepare feature vectors

In [659]:
# compute the mean word2vec feature of each review as input
def getFeature(text):
    words = text.split()
    vectors = []
    for word in words:
        if word in wv:
            vectors.append(wv[word])
    vector = np.mean(vectors, axis=0)
    return vector

train = train_data.copy()
test = test_data.copy()
train['input'] = train['review'].apply(getFeature)
test['input'] = test['review'].apply(getFeature)
train = train.drop(columns='review')
test = test.drop(columns='review')
train = train.dropna(axis = 0, how = 'any')
test = test.dropna(axis = 0, how = 'any')

/Users/libin/opt/miniconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/libin/opt/miniconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,


In [660]:
X_train = train['input'].to_numpy().tolist()
X_test = test['input'].to_numpy().tolist()
Y_train = train['star_rating']
Y_test = test['star_rating']

## (b) Perceprton

In [607]:
from sklearn.linear_model import Perceptron
from sklearn.metrics import accuracy_score

clf = Perceptron(max_iter=8000, alpha= 0.1, early_stopping=True)

clf.fit(X_train, Y_train)

Y_pred = clf.predict(X_test)

perceptron_accuracy = accuracy_score(Y_test, Y_pred)
print('The accuracy of Perceptron is ', perceptron_accuracy)

The accuracy of Perceptron is  0.5932415519399249


## (c) SVM

In [608]:
from sklearn.svm import LinearSVC

clf = LinearSVC(C=0.1, max_iter=2000)

clf.fit(X_train, Y_train)

Y_pred = clf.predict(X_test)

svm_accuracy = accuracy_score(Y_test, Y_pred)
print('The accuracy of SVM is ', svm_accuracy)

The accuracy of SVM is  0.6593241551939925


# Task 4. Forward Neural Networks

#### Note: When tuning all the following neural networks, I tried to tune the activation functions,
#### number of batches, number of epochs using val set, learning rate, and GD methods.
#### I found the performance of the networks is highly depend on the sampled data, even though I sampled data
#### in the same way, they can give very different results. So I mainly focused on comparison of the networks.

## (a) Using average word2vec

In [661]:
# build the Feedforward Neural Network structure
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        # number of nodes in each layer
        hidden_1 = 100
        hidden_2 = 10

        self.fc1 = nn.Linear(300, hidden_1)
        self.fc2 = nn.Linear(hidden_1, hidden_2)
        self.fc3 = nn.Linear(hidden_2, 3)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.softmax(x, dim=1) # use softmax to generate output

# initialize the NN
model = Net()

In [662]:
# use cross-entropy loss
criterion = nn.CrossEntropyLoss()

# use Adam gradient descent and learning rate = 0.001
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [663]:
# prepare data, DataLoader
from torch.utils.data import DataLoader, Dataset

new_train = train.copy()
new_test = test.copy()

# modify the label to match the NN's indexing
new_train['star_rating'] = new_train['star_rating'] - 1
new_test['star_rating'] = new_test['star_rating'] - 1

class MyData(Dataset):
    def __init__(self, dataframe):
        self.input = dataframe['input'].values
        self.output = dataframe['star_rating'].values

        self.length = len(self.output)

    def __getitem__(self, index):
        return self.input[index], self.output[index]

    def __len__(self):
        return self.length

training = MyData(new_train)
testing = MyData(new_test)

train_loader = DataLoader(dataset=training,batch_size=32, shuffle=True)
test_loader = DataLoader(dataset=testing, shuffle=False)

In [664]:
# train the model

n_epochs = 50 # number of epochs to train the model

for epoch in range(n_epochs):
    # monitor training loss
    train_loss = 0.0

    model.train() # prep model for training
    for data, target in train_loader:

        optimizer.zero_grad()

        output = model(data)

        loss = criterion(output, target)

        loss.backward()
        optimizer.step()
        train_loss += loss.item()*data.size(0)
    # compute loss
    train_loss = train_loss/len(train_loader.dataset)

    print('Epoch: {} \tTraining Loss: {:.6f}'.format(epoch+1, train_loss,))


Epoch: 1 	Training Loss: 0.939405
Epoch: 2 	Training Loss: 0.889882
Epoch: 3 	Training Loss: 0.883836
Epoch: 4 	Training Loss: 0.879370
Epoch: 5 	Training Loss: 0.875800
Epoch: 6 	Training Loss: 0.874333
Epoch: 7 	Training Loss: 0.871841
Epoch: 8 	Training Loss: 0.868916
Epoch: 9 	Training Loss: 0.867375
Epoch: 10 	Training Loss: 0.866241
Epoch: 11 	Training Loss: 0.863411
Epoch: 12 	Training Loss: 0.861644
Epoch: 13 	Training Loss: 0.859459
Epoch: 14 	Training Loss: 0.858097
Epoch: 15 	Training Loss: 0.855882
Epoch: 16 	Training Loss: 0.853982
Epoch: 17 	Training Loss: 0.852472
Epoch: 18 	Training Loss: 0.850258
Epoch: 19 	Training Loss: 0.848851
Epoch: 20 	Training Loss: 0.845609
Epoch: 21 	Training Loss: 0.844023
Epoch: 22 	Training Loss: 0.843476
Epoch: 23 	Training Loss: 0.840219
Epoch: 24 	Training Loss: 0.838527
Epoch: 25 	Training Loss: 0.836811
Epoch: 26 	Training Loss: 0.834085
Epoch: 27 	Training Loss: 0.832646
Epoch: 28 	Training Loss: 0.830992
Epoch: 29 	Training Loss: 0.8

In [666]:
# predict using test set and calculate accuracy
count = 0
for data, target in test_loader:
    data = data.to(torch.float32)
    output = model(data)
    _, pred = torch.max(output, 1)
    if pred == target:
        count += 1
print('The accuracy of MLP using average word2vec is ',count/len(test_loader.dataset))

The accuracy of MLP using average word2vec is  0.657488527325824


## (b) Using concat feature

In [672]:
# compute the concatenate word2vec feature of each review as input
def getConcatFeature(text):
    words = text.split()
    vector = []
    count = 0
    for word in words:
        if word in wv:
            vector.extend(wv[word].tolist())
            count += 1
        if count == 10:
            break
    if count != 10:
        for i in range(10 - count):
            newVec = [0 for i in range(300)]
            vector.extend(newVec)

    return np.array(vector)

train_4b = train_data.copy()
test_4b = test_data.copy()
train_4b['input'] = train_4b['review'].apply(getConcatFeature)
test_4b['input'] = test_4b['review'].apply(getConcatFeature)
train_4b = train_4b.drop(columns='review')
test_4b = test_4b.drop(columns='review')
train_4b = train_4b.dropna(axis = 0, how = 'any')
test_4b = test_4b.dropna(axis = 0, how = 'any')

In [683]:
# build the Feedforward Neural Network structure
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        hidden_1 = 100
        hidden_2 = 10

        self.fc1 = nn.Linear(3000, hidden_1)
        self.fc2 = nn.Linear(hidden_1, hidden_2)
        self.fc3 = nn.Linear(hidden_2, 3)


    def forward(self, x):

        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.softmax(x, dim=1)

# initialize the NN
model = Net()

In [684]:
# use cross-entropy
criterion = nn.CrossEntropyLoss()

# use Adam gradient descent and learning rate = 0.001
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

In [685]:
# prepare data and DataLoader
new_train = train_4b.copy()
new_test = test_4b.copy()
new_train['star_rating'] = new_train['star_rating'] - 1
new_test['star_rating'] = new_test['star_rating'] - 1
class MyData(Dataset):
    def __init__(self, dataframe):
        self.input = dataframe['input'].values
        self.output = dataframe['star_rating'].values

        self.length = len(self.output)

    def __getitem__(self, index):
        return self.input[index], self.output[index]

    def __len__(self):
        return self.length

training = MyData(new_train)
testing = MyData(new_test)

train_loader = DataLoader(dataset=training, shuffle=True)
test_loader = DataLoader(dataset=testing, shuffle=False)

In [686]:
# train the model
n_epochs = 12 # number of epochs to train the model

for epoch in range(n_epochs):
    # monitor training loss
    train_loss = 0.0


    model.train()
    for data, target in train_loader:
        data = data.to(torch.float32)

        optimizer.zero_grad()

        output = model(data)

        loss = criterion(output, target)

        loss.backward()

        optimizer.step()

        train_loss += loss.item()*data.size(0)

    train_loss = train_loss/len(train_loader.dataset)

    '''
    model.eval()
    count = 0
    for data, target in test_loader:
        data = data.to(torch.float32)
        output = model(data)
        _, pred = torch.max(output, 1)
        if pred == target:
            count += 1
    print(count/len(test_loader.dataset))
    '''

    print('Epoch: {} \tTraining Loss: {:.6f}'.format(
        epoch+1,
        train_loss,
        #valid_loss
        ))

    # save model if validation loss has decreased

Epoch: 1 	Training Loss: 1.097377
Epoch: 2 	Training Loss: 1.066973
Epoch: 3 	Training Loss: 0.993504
Epoch: 4 	Training Loss: 0.961152
Epoch: 5 	Training Loss: 0.940906
Epoch: 6 	Training Loss: 0.928071
Epoch: 7 	Training Loss: 0.919188
Epoch: 8 	Training Loss: 0.911365
Epoch: 9 	Training Loss: 0.905841
Epoch: 10 	Training Loss: 0.900504
Epoch: 11 	Training Loss: 0.895248
Epoch: 12 	Training Loss: 0.891395


In [687]:
# predict on test set and calculate accuracy
count = 0
for data, target in test_loader:
    data = data.to(torch.float32)
    output = model(data)
    _, pred = torch.max(output, 1)
    if pred == target:
        count += 1
print('The accuracy of MLP using concat word2vec is',count/len(test_loader.dataset))

The accuracy of MLP using concat word2vec is 0.5648333333333333


# Task 5. Recurrent Neural Network

## (a) RNN

In [688]:
# compute sequential word2vec feature of each review as input
def getRNNFeature(text):
    words = text.split()
    vector = []
    count = 0
    for word in words:
        if word in wv:
            vector.append(wv[word].tolist())
            count += 1
        if count == 20:
            break
    if count != 20:
        for i in range(20 - count):
            newVec = [0 for i in range(300)]
            vector.append(newVec)

    return np.array(vector)

train_5a = train_data.copy()
test_5a = test_data.copy()
train_5a['input'] = train_5a['review'].apply(getRNNFeature)
test_5a['input'] = test_5a['review'].apply(getRNNFeature)
train_5a = train_5a.drop(columns='review')
test_5a = test_5a.drop(columns='review')
train_5a = train_5a.dropna(axis = 0, how = 'any')
test_5a = test_5a.dropna(axis = 0, how = 'any')

In [689]:
# prepare data and DataLoader
rnn_train = train_5a.copy()
rnn_test = test_5a.copy()
rnn_train['star_rating'] = rnn_train['star_rating'] - 1
rnn_test['star_rating'] = rnn_test['star_rating'] - 1
class MyData(Dataset):
    def __init__(self, dataframe):
        self.input = dataframe['input'].values
        self.output = dataframe['star_rating'].values

        self.length = len(self.output)

    def __getitem__(self, index):
        return self.input[index], self.output[index]

    def __len__(self):
        return self.length

training = MyData(rnn_train)
testing = MyData(rnn_test)

train_loader = DataLoader(dataset=training,batch_size=32, shuffle=True)
test_loader = DataLoader(dataset=testing, shuffle=False)

In [648]:
# build the RNN structure
class RNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, layer_num):
        super(RNN, self).__init__()

        self.layer_num = layer_num

        self.hidden_dim = hidden_dim

        self.rnn = nn.RNN(input_dim, hidden_dim, num_layers=self.layer_num, batch_first=True, nonlinearity='relu')

        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        h0 = torch.zeros(self.layer_num, x.size(0), self.hidden_dim).requires_grad_()
        output, _ = self.rnn(x, h0.detach())
        output = output[:,-1,:]
        output = F.relu(output)
        output = self.fc(output)
        return output

    def initHidden(self):
        return torch.zeros(1, self.hidden_dim)


rnn_model = RNN(300, 20, 3, 2)

In [649]:
# use CrossEntropyLoss
criterion = nn.CrossEntropyLoss()

# use Adam GD and learning rate = 0.0005
optimizer = torch.optim.Adam(rnn_model.parameters(), lr=0.0005)

In [650]:
# train the model
n_epochs = 20

for epoch in range(n_epochs):
    train_loss = 0.0

    rnn_model.train()
    for data, target in train_loader:
        data = data.to(torch.float32)
        optimizer.zero_grad()

        output = rnn_model(data)

        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*data.size(0)


    train_loss = train_loss/len(train_loader.dataset)

    '''
    count = 0
    rnn_model.eval()
    for data, target in test_loader:
        data = data.to(torch.float32)
        output = rnn_model(data)
        _, pred = torch.max(output, 1)
        if pred == target:
            count += 1
    print('accuracy on test is: ', count/len(test_loader.dataset))
    '''


    print('Epoch: {} \tTraining Loss: {:.6f}'.format(epoch+1, train_loss))

Epoch: 1 	Training Loss: 1.031782
Epoch: 2 	Training Loss: 0.878597
Epoch: 3 	Training Loss: 0.837002
Epoch: 4 	Training Loss: 0.812297
Epoch: 5 	Training Loss: 0.797108
Epoch: 6 	Training Loss: 0.783459
Epoch: 7 	Training Loss: 0.774925
Epoch: 8 	Training Loss: 0.760690
Epoch: 9 	Training Loss: 0.751393
Epoch: 10 	Training Loss: 0.742999
Epoch: 11 	Training Loss: 0.736459
Epoch: 12 	Training Loss: 0.730472
Epoch: 13 	Training Loss: 0.726213
Epoch: 14 	Training Loss: 0.720817
Epoch: 15 	Training Loss: 0.717068
Epoch: 16 	Training Loss: 0.713203
Epoch: 17 	Training Loss: 0.708905
Epoch: 18 	Training Loss: 0.705315
Epoch: 19 	Training Loss: 0.702833
Epoch: 20 	Training Loss: 0.698327


In [651]:
# predict using the test set and calculate accuracy
count = 0
rnn_model.eval()
for data, target in test_loader:
    data = data.to(torch.float32)
    output = rnn_model(data)
    _, pred = torch.max(output, 1)
    if pred == target:
        count += 1
print('The accuracy of RNN on test set is ', count/len(test_loader.dataset))

The accuracy of RNN on test set is  0.6511666666666667


## (b) GRU

In [690]:
# build the RNN with GRU cells
class GRU(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, layer_num):
        super(GRU, self).__init__()

        self.layer_num = layer_num

        self.hidden_dim = hidden_dim

        self.rnn = nn.GRU(input_dim, hidden_dim, num_layers=self.layer_num, batch_first=True)

        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        h0 = torch.zeros(self.layer_num, x.size(0), self.hidden_dim).requires_grad_()
        output, _ = self.rnn(x, h0.detach())
        output = output[:,-1,:]
        output = F.relu(output)
        output = self.fc(output)
        return output

    def initHidden(self):
        return torch.zeros(1, self.hidden_dim)


gru_model = GRU(300, 20, 3, 2)

In [691]:
# use CrossEntropyLoss
criterion = nn.CrossEntropyLoss()

# use Adam GD and learning rate = 0.0005
optimizer = torch.optim.Adam(gru_model.parameters(), lr=0.0005)

In [692]:
# train the model
n_epochs = 22

for epoch in range(n_epochs):
    train_loss = 0.0

    gru_model.train()
    for data, target in train_loader:
        data = data.to(torch.float32)
        optimizer.zero_grad()

        output = gru_model(data)

        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*data.size(0)


    train_loss = train_loss/len(train_loader.dataset)
    '''
    count = 0
    gru_model.eval()
    for data, target in test_loader:
        data = data.to(torch.float32)
        output = gru_model(data)
        _, pred = torch.max(output, 1)
        if pred == target:
            count += 1
    print('accuracy on test is: ', count/len(test_loader.dataset))
    '''


    print('Epoch: {} \tTraining Loss: {:.6f}'.format(epoch+1, train_loss))

Epoch: 1 	Training Loss: 0.957697
Epoch: 2 	Training Loss: 0.846718
Epoch: 3 	Training Loss: 0.796572
Epoch: 4 	Training Loss: 0.770886
Epoch: 5 	Training Loss: 0.753284
Epoch: 6 	Training Loss: 0.741363
Epoch: 7 	Training Loss: 0.729519
Epoch: 8 	Training Loss: 0.721023
Epoch: 9 	Training Loss: 0.711605
Epoch: 10 	Training Loss: 0.704843
Epoch: 11 	Training Loss: 0.699004
Epoch: 12 	Training Loss: 0.691164
Epoch: 13 	Training Loss: 0.684915
Epoch: 14 	Training Loss: 0.679431
Epoch: 15 	Training Loss: 0.674279
Epoch: 16 	Training Loss: 0.668736
Epoch: 17 	Training Loss: 0.664794
Epoch: 18 	Training Loss: 0.658272
Epoch: 19 	Training Loss: 0.653814
Epoch: 20 	Training Loss: 0.650249
Epoch: 21 	Training Loss: 0.646058
Epoch: 22 	Training Loss: 0.642572


In [693]:
# predict using the test set and calculate accuracy
count = 0
gru_model.eval()
for data, target in test_loader:
    data = data.to(torch.float32)
    output = gru_model(data)
    _, pred = torch.max(output, 1)
    if pred == target:
        count += 1
print('The accuracy of GRU on test set is ',count/len(test_loader.dataset))

The accuracy of GRU on test set is  0.6669166666666667


## (c) LSTM

In [705]:
# build the RNN with LSTM cells
class LSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, layer_num):
        super(LSTM, self).__init__()

        self.layer_num = layer_num

        self.hidden_dim = hidden_dim

        self.rnn = nn.LSTM(input_dim, hidden_dim, num_layers=self.layer_num, batch_first=True)

        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        #h0 = torch.zeros(self.layer_num, x.size(0), self.hidden_dim).requires_grad_()
        h0 = torch.zeros(self.layer_num, x.size(0), self.hidden_dim).requires_grad_()
        c0 = torch.zeros(self.layer_num, x.size(0), self.hidden_dim).requires_grad_()

        output, _ = self.rnn(x, (h0.detach(),c0.detach()))
        output = output[:,-1,:]
        output = F.relu(output)
        output = self.fc(output)
        return output

    def initHidden(self):
        return torch.zeros(1, self.hidden_dim)


lstm_model = LSTM(300, 20, 3, 2)

In [706]:
# use CrossEntropyLoss
criterion = nn.CrossEntropyLoss()

# use Adam GD and learning rate = 0.0005
optimizer = torch.optim.Adam(lstm_model.parameters(), lr=0.0005)

In [707]:
n_epochs = 30

for epoch in range(n_epochs):
    train_loss = 0.0

    lstm_model.train()
    for data, target in train_loader:
        data = data.to(torch.float32)
        optimizer.zero_grad()

        output = lstm_model(data)

        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*data.size(0)


    train_loss = train_loss/len(train_loader.dataset)
    '''
    count = 0
    lstm_model.eval()
    for data, target in test_loader:
        data = data.to(torch.float32)
        output = lstm_model(data)
        _, pred = torch.max(output, 1)
        if pred == target:
            count += 1
    print('accuracy on test is: ', count/len(test_loader.dataset))
    '''


    print('Epoch: {} \tTraining Loss: {:.6f}'.format(epoch+1, train_loss))

Epoch: 1 	Training Loss: 0.965559
Epoch: 2 	Training Loss: 0.856667
Epoch: 3 	Training Loss: 0.814796
Epoch: 4 	Training Loss: 0.789789
Epoch: 5 	Training Loss: 0.771713
Epoch: 6 	Training Loss: 0.756592
Epoch: 7 	Training Loss: 0.743536
Epoch: 8 	Training Loss: 0.731910
Epoch: 9 	Training Loss: 0.723249
Epoch: 10 	Training Loss: 0.713810
Epoch: 11 	Training Loss: 0.705455
Epoch: 12 	Training Loss: 0.697131
Epoch: 13 	Training Loss: 0.688954
Epoch: 14 	Training Loss: 0.682845
Epoch: 15 	Training Loss: 0.675805
Epoch: 16 	Training Loss: 0.669001
Epoch: 17 	Training Loss: 0.664491
Epoch: 18 	Training Loss: 0.658675
Epoch: 19 	Training Loss: 0.652997
Epoch: 20 	Training Loss: 0.649106
Epoch: 21 	Training Loss: 0.644366
Epoch: 22 	Training Loss: 0.638345
Epoch: 23 	Training Loss: 0.634089
Epoch: 24 	Training Loss: 0.628480
Epoch: 25 	Training Loss: 0.626207
Epoch: 26 	Training Loss: 0.621550
Epoch: 27 	Training Loss: 0.617512
Epoch: 28 	Training Loss: 0.612027
Epoch: 29 	Training Loss: 0.6

In [708]:
# predict using the test set and calculate accuracy
count = 0
lstm_model.eval()
for data, target in test_loader:
    data = data.to(torch.float32)
    output = lstm_model(data)
    _, pred = torch.max(output, 1)
    if pred == target:
        count += 1
print('The accuracy of LSTM on test set is ', count/len(test_loader.dataset))

The accuracy of LSTM on test set is  0.66125
